<a href="https://colab.research.google.com/github/marianasod/xp_projeto_pratico/blob/main/Jun%C3%A7%C3%A3o_dos_dados_a%C3%A7%C3%B5es_e_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import date
import spacy
import re
from transformers import pipeline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from google.colab import files


Funções


In [ ]:
def resumo_sentimentos_data(df:pd.DataFrame, groupped_data: list):
    """
    Função para contar a frequência de cada sentimento em um determinado dia.

    Parâmetros:
    df (DataFrame): DataFrame contendo os dados.
    groupped_data (list): Coluna contendo a data de referência.
    sentimento (str): Sentimento a ser contabilizado.
    """

    # Agrupando por dia e contando a frequência de cada sentimento
    sentimento_contagem = df.groupby(groupped_data).size().unstack(fill_value=0)
    sentimento_contagem['sentimento_mais_frequente'] = sentimento_contagem.idxmax(axis=1)
    df_final = sentimento_contagem[['sentimento_mais_frequente']].reset_index()
    return df_final

def fill_sentiments_date(df, start_date, end_date , date_format, sentimento):
    # Converter a coluna de data para o formato datetime
    df[date_format] = pd.to_datetime(df[date_format])

    # range completo de datas
    full_date_range =  pd.DataFrame(pd.date_range(start=start_date, end=end_date), columns=[date_format])
    merged_df = pd.merge(full_date_range, df_final_day[[date_format, sentimento]], on=date_format, how="left")
    merged_df[sentimento] = merged_df[sentimento].fillna(method='ffill')
    return merged_df

def format_date(date_str):
    # Lista de possíveis formatos de entrada
    date_formats = [
        '%Y-%m-%d %H:%M:%S',       # Ex: '2024-07-24 00:00:00'
        '%a, %d %b %Y %H:%M:%S',    # Ex: 'Thu, 02 May 2024 07:00:00'
    ]

    for fmt in date_formats:
        try:
            # Tentativa de parsing usando o formato atual
            date_obj = datetime.strptime(date_str, fmt)
            return date_obj.strftime('%Y-%m-%d')
        except ValueError:
            continue

# Função para converter a data
def convert_date(date_str):
    date_str = str(date_str)
    date_str = date_str.replace(" GMT", "")

    try:
        date_obj = datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S')
    except:
        # Tentativa de parsing do segundo formato
        date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

    return date_obj.strftime('%Y-%m-%d')

def convert_date_ano_mes(date_str):
    date_str = str(date_str)
    date_str = date_str.replace(" GMT", "")

    try:
        date_obj = datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S')
    except:
        # Tentativa de parsing do segundo formato
        date_obj = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')

    #return datetime.strptime(date_str, '%a, %d %b %Y %H:%M:%S').strftime('%Y-%m-%d')
    return date_obj.strftime('%Y-%m')

Leitura das bases

In [ ]:
precos = pd.read_excel("/content/precos_acoes.xlsx")

In [ ]:
sentimentos = pd.read_excel("/content/base_sentimentos_dataset_completo_2023_2024.xlsx")

In [ ]:
precos.head()

,Data,Preco_close,ticker,coluna_preco_scaled
0,2022-08-16,14.093428,BBDC3.SA,0.776189
1,2022-08-17,14.145307,BBDC3.SA,0.786547
2,2022-08-18,14.205832,BBDC3.SA,0.798630
3,2022-08-19,14.015610,BBDC3.SA,0.760654
4,2022-08-22,13.859978,BBDC3.SA,0.729583


In [ ]:
sentimentos.head()

,title,description,published date,url,publisher,term,noticia_composta,noticia_processada,date_format,Ano_Mes,sentimento_1,pontuacao_1,sentimento_2,pontuacao_2,sentimento_3,pontuacao_3,sentimento_final
0,Bradesco tem leve queda em lucro do 1º trimest...,Bradesco tem leve queda em lucro do 1º trimest...,"Thu, 02 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMilwFBV...,"{'href': 'https://forbes.com.br', 'title': 'Fo...",Bradesco,Bradesco tem leve queda em lucro do 1º trimest...,bradesco tem leve queda em lucro do trimestre...,2024-05-02,2024-05,neutral,0.475062,negative,0.454790,Neutral,0.965160,neutral
1,Bradesco avalia dez agências para atendimento ...,Bradesco avalia dez agências para atendimento ...,"Tue, 30 Jul 2024 20:58:00 GMT",https://news.google.com/rss/articles/CBMitwFBV...,"{'href': 'https://www.meioemensagem.com.br', '...",Bradesco,Bradesco avalia dez agências para atendimento ...,bradesco avalia dez agências para atendimento ...,2024-07-30,2024-07,neutral,0.862426,positive,0.663487,Neutral,0.989767,neutral
2,"Marcelo Noronha, presidente do Bradesco: “O ce...","Marcelo Noronha, presidente do Bradesco: “O ce...","Fri, 31 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMimgFBV...,"{'href': 'https://veja.abril.com.br', 'title':...",Bradesco,"Marcelo Noronha, presidente do Bradesco: “O ce...",marcelo noronha presidente do bradesco o cenár...,2024-05-31,2024-05,neutral,0.486094,positive,0.918528,Neutral,0.980144,neutral
3,"Em nova campanha, Bradesco revela universo de ...","Em nova campanha, Bradesco revela universo de ...","Tue, 30 Apr 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMi7wFBV...,"{'href': 'https://www.folhavitoria.com.br', 't...",Bradesco,"Em nova campanha, Bradesco revela universo de ...",em nova campanha bradesco revela universo de p...,2024-04-30,2024-04,positive,0.571874,positive,0.834235,Neutral,0.980506,positive
4,"Bradesco vê melhora, ainda lenta, no 1º result...","Bradesco vê melhora, ainda lenta, no 1º result...","Thu, 02 May 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMijwFBV...,"{'href': 'https://braziljournal.com', 'title':...",Bradesco,"Bradesco vê melhora, ainda lenta, no 1º result...",bradesco vê melhora ainda lenta no resultado ...,2024-05-02,2024-05,neutral,0.519785,positive,0.747271,Neutral,0.978800,neutral


Resumo sentimentos data

In [ ]:
df_final_day = resumo_sentimentos_data(sentimentos, ['date_format','sentimento_final'])

In [ ]:
merged_df = fill_sentiments_date(df_final_day, "2023-06-01", "2024-07-31", "date_format", 'sentimento_mais_frequente')
merged_df.head()

<ipython-input-8-6c633fd0e12b>:24: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  merged_df[sentimento] = merged_df[sentimento].fillna(method='ffill')


,date_format,sentimento_mais_frequente
0,2023-06-01,neutral
1,2023-06-02,neutral
2,2023-06-03,negative
3,2023-06-04,neutral
4,2023-06-05,neutral


Juntando os conjuntos de dados de preços e sentimentos

In [ ]:
df_precos_sentimentos = precos.merge(merged_df, left_on='Data', right_on='date_format', how = "inner")
df_precos_sentimentos = df_precos_sentimentos[["Data", "Preco_close", "coluna_preco_scaled", "sentimento_mais_frequente"]]

In [ ]:
df_precos_sentimentos.head()

,Data,Preco_close,coluna_preco_scaled,sentimento_mais_frequente
0,2023-06-01,12.184021,0.394995,neutral
1,2023-06-02,12.199749,0.398135,neutral
2,2023-06-05,12.664763,0.490970,neutral
3,2023-06-06,12.929182,0.543759,neutral
4,2023-06-07,12.965654,0.551040,neutral


In [ ]:
### salvando o dataframe
# Exportar o DataFrame para um arquivo Excel
file_name = 'base_sentimentos_noticias.xlsx'
df_precos_sentimentos.to_excel(file_name, index=False, engine='openpyxl')

# Download do arquivo Excel
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>